# Test performances

## Test data I/O

### Test dense data

In [1]:
import importlib
import numpy as np
import nibabel as nib
import tensorflow as tf

from utils import is_equal, load_data
from utils.med_utils import data_io, pre_processing, load_medical_data

path = '/storage/Totalsegmentator_dataset/s0001/ct'

data = nib.load(path + '.nii.gz')
val  = data.get_fdata('unchanged')

np.save(path + '-1.5-1.5-1.5.npy', val)
np.savez(path + '.npz', data = val, pixdim = data.header['pixdim'][1:4])
tf.io.write_file(path + '-1.5-1.5-1.5.tensor', tf.io.serialize_tensor(tf.cast(val, tf.float32)))

2023-06-12 09:49:20.203886: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 09:49:20.304351: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-12 09:49:20.328509: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. E

In [2]:
importlib.reload(pre_processing)
importlib.reload(data_io)

tf_load_medical_data = tf.function(data_io.load_medical_data)

test_kwargs = [
    {},
    {'slice_start' : tf.cast(3, tf.int32)},
    {'slice_end' : tf.cast(6, tf.int32)},
    {'slice_start' : tf.cast(3, tf.int32), 'slice_end' : tf.cast(6, tf.int32)},
    {'slice_start' : tf.cast(3, tf.int32), 'slice_end' : tf.cast(6, tf.int32), 'slice_axis' : tf.cast(0, tf.int32)}
]
for kwargs in test_kwargs:
    print('\nTest with kwargs : {}'.format(kwargs))
    r1 = data_io.load_medical_data(path + '.nii.gz', ** kwargs)
    r2 = data_io.load_medical_data(path + '-1.5-1.5-1.5.npy', ** kwargs)
    r3 = data_io.load_medical_data(path + '.npz', ** kwargs)
    r4 = data_io.load_medical_data(path + '-1.5-1.5-1.5.tensor', ** kwargs)

    r5 = data_io.load_medical_data(tf.cast(path + '.nii.gz', tf.string), ** kwargs)
    r6 = data_io.load_medical_data(tf.cast(path + '-1.5-1.5-1.5.npy', tf.string), ** kwargs)
    r7 = data_io.load_medical_data(tf.cast(path + '.npz', tf.string), ** kwargs)
    r8 = data_io.load_medical_data(tf.cast(path + '-1.5-1.5-1.5.tensor', tf.string), ** kwargs)

    r9  = tf_load_medical_data(tf.cast(path + '.nii.gz', tf.string), ** kwargs)
    r10 = tf_load_medical_data(tf.cast(path + '-1.5-1.5-1.5.npy', tf.string), ** kwargs)
    r11 = tf_load_medical_data(tf.cast(path + '.npz', tf.string), ** kwargs)
    r12 = tf_load_medical_data(tf.cast(path + '-1.5-1.5-1.5.tensor', tf.string), ** kwargs)

    print(is_equal(r1, r2)[1])
    print(is_equal(r1, r3)[1])
    print(is_equal(r1, r4)[1])

    print(is_equal(r1, r5)[1])
    print(is_equal(r1, r6)[1])
    print(is_equal(r1, r7)[1])
    print(is_equal(r1, r8)[1])

    print(is_equal(r1, r9)[1])
    print(is_equal(r1, r10)[1])
    print(is_equal(r1, r11)[1])
    print(is_equal(r1, r12)[1])



Test with kwargs : {}
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !

Test with kwargs : {'slice_start': <tf.Tensor: shape=(), dtype=int32, numpy=3>}
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !

Test with kwargs : {'slice_end': <tf.Tensor: shape=(), dtype=int32, numpy=6>}
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !

Test with kwargs : {'slice_start': <tf.Tensor: shape=(), dtype=int32, numpy=3>, 'slice_end': <tf.Tensor: shape=(), dtype=int32, numpy=6>}
Value are equals !
Value are equals !
Value are equ

In [3]:
%timeit load_medical_data(path + '.nii.gz')
%timeit load_medical_data(path + '-1.5-1.5-1.5.npy')
%timeit load_medical_data(path + '.npz')
%timeit load_medical_data(path + '-1.5-1.5-1.5.tensor')
print()
%timeit load_medical_data(tf.cast(path + '.nii.gz', tf.string))
%timeit load_medical_data(tf.cast(path + '-1.5-1.5-1.5.npy', tf.string))
%timeit load_medical_data(tf.cast(path + '.npz', tf.string))
%timeit load_medical_data(tf.cast(path + '-1.5-1.5-1.5.tensor', tf.string))
print()
%timeit tf_load_medical_data(tf.cast(path + '.nii.gz', tf.string))
%timeit tf_load_medical_data(tf.cast(path + '-1.5-1.5-1.5.npy', tf.string))
%timeit tf_load_medical_data(tf.cast(path + '.npz', tf.string))
%timeit tf_load_medical_data(tf.cast(path + '-1.5-1.5-1.5.tensor', tf.string))

182 ms ± 57.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
25 ms ± 174 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
141 ms ± 37 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
32 ms ± 148 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

226 ms ± 175 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
69.4 ms ± 340 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
186 ms ± 189 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
32.1 ms ± 167 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

232 ms ± 674 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
74.2 ms ± 519 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
190 ms ± 243 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
31.4 ms ± 218 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Test sparse data

In [1]:
import os
import importlib
import numpy as np
import nibabel as nib
import tensorflow as tf

from utils import is_equal, load_data
from utils.med_utils.label_utils import TOTALSEGMENTATOR_LABELS
from utils.med_utils import data_io, pre_processing, label_utils, sparse_utils, load_medical_data

tf.config.set_visible_devices([], 'GPU')

path = '/storage/Totalsegmentator_dataset/s0001/masks'

labels = sorted([f[:-7] for f in os.listdir(path[:-5] + 'segmentations/')])

data = nib.load(path + '.nii.gz')
val  = data.get_fdata('unchanged')

print('Converting to sparse')
path_infos = '-1.5-1.5-1.5-{}'.format('-'.join([str(s) for s in data.shape]))
with tf.device('cpu'):
    tensor = tf.cast(val, tf.uint8)
    sparse = tf.sparse.from_dense(tensor)

print('Saving...')
%time np.save(path + path_infos + '.npy', val)
#%time np.savez(path + '.npz', data = val, pixdim = data.header['pixdim'][1:4], labels = labels)
%time np.savez(path + '-sparse.npz', mask = sparse.indices, pixdim = data.header['pixdim'][1:4], labels = labels, shape = val.shape)
#%time tf.io.write_file(path + path_infos + '.tensor', tf.io.serialize_tensor(tensor))
%time tf.io.write_file(path + path_infos + '.stensor', tf.io.serialize_tensor(sparse.indices))

2023-06-12 09:41:14.336458: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 09:41:14.433925: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-12 09:41:14.458382: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. E

Converting to sparse


2023-06-12 09:41:25.698124: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Saving...
CPU times: user 0 ns, sys: 2.61 s, total: 2.61 s
Wall time: 2.62 s
CPU times: user 41 ms, sys: 30.9 ms, total: 71.9 ms
Wall time: 71.9 ms
CPU times: user 26.4 ms, sys: 31.3 ms, total: 57.7 ms
Wall time: 57.8 ms


In [3]:
importlib.reload(sparse_utils)
importlib.reload(label_utils)
importlib.reload(pre_processing)
importlib.reload(data_io)

def _to_dense(res):
    sp, v = res
    return tf.sparse.to_dense(sp), v

sp_kwargs = {'dtype' : tf.uint8, 'use_sparse' : True} #tf.SparseTensorSpec(shape = (None, None, None, None), dtype = tf.uint8)

tf_load_medical_data = tf.function(data_io.load_medical_data)

test_kwargs = [
    {},
    {'slice_end' : tf.cast(6, tf.int32)},
    {'slice_start' : tf.cast(3, tf.int32), 'slice_end' : tf.cast(6, tf.int32)},
    {'slice_start' : tf.cast(3, tf.int32), 'slice_end' : tf.cast(6, tf.int32), 'slice_axis' : tf.cast(0, tf.int32)},
    {
        'slice_start' : tf.cast(8, tf.int32), 'slice_end' : tf.cast(64, tf.int32),
        'labels' : labels, 'mapping' : [None] + labels
    },
    {
        'slice_start' : tf.cast(8, tf.int32), 'slice_end' : tf.cast(64, tf.int32),
        'labels' : labels, 'mapping' : TOTALSEGMENTATOR_LABELS
    },
    {
        'slice_start' : tf.cast(8, tf.int32), 'slice_end' : tf.cast(64, tf.int32),
        'labels' : tf.cast(labels, tf.string), 'mapping' : [None] + labels
    },
    {
        'slice_start' : tf.cast(8, tf.int32), 'slice_end' : tf.cast(64, tf.int32),
        'labels' : tf.cast(labels, tf.string), 'mapping' : label_utils.build_mapping(TOTALSEGMENTATOR_LABELS, 'tensor')
    },
    {
        'slice_start' : tf.cast(8, tf.int32), 'slice_end' : tf.cast(64, tf.int32), 'labels' : tf.cast(labels, tf.string),
        'mapping' : label_utils.build_mapping([None, ('heart_atrium_left', 'heart_atrium_right'), 'aorta'], 'tensor')
    }
][1:]
for kwargs in test_kwargs:
    print('\nTest with kwargs : {}'.format(kwargs))
    r1 = data_io.load_medical_data(path + '.nii.gz', dtype = tf.uint8, ** kwargs)

    r13 = tf_load_medical_data(tf.cast(path + '.nii.gz', tf.string), dtype = tf.uint8, ** kwargs)
    r14 = tf_load_medical_data(tf.cast(path + path_infos + '.npy', tf.string), dtype = tf.uint8, ** kwargs)
    #r15 = tf_load_medical_data(tf.cast(path + '.npz', tf.string), dtype = tf.uint8, ** kwargs)
    r16 = _to_dense(tf_load_medical_data(tf.cast(path + '-sparse.npz', tf.string), ** sp_kwargs, ** kwargs))
    #r17 = tf_load_medical_data(tf.cast(path + path_infos + '.tensor', tf.string), dtype = tf.uint8, ** kwargs)
    r18 = _to_dense(tf_load_medical_data(tf.cast(path + path_infos + '.stensor', tf.string), ** sp_kwargs, ** kwargs))

    print(is_equal(r1, r13)[1])
    print(is_equal(r1, r14)[1])
    #print(is_equal(r1, r15)[1])
    print(is_equal(r1, r16)[1])
    #print(is_equal(r1, r17)[1])
    print(is_equal(r1, r18)[1])

    r2 = data_io.load_medical_data(path + path_infos + '.npy', dtype = tf.uint8, ** kwargs)
    #r3 = data_io.load_medical_data(path + '.npz', dtype = tf.uint8, ** kwargs)
    r4 = _to_dense(data_io.load_medical_data(path + '-sparse.npz', ** sp_kwargs, ** kwargs))
    #r5 = data_io.load_medical_data(path + path_infos + '.tensor', dtype = tf.uint8, ** kwargs)
    r6 = _to_dense(data_io.load_medical_data(path + path_infos + '.stensor', ** sp_kwargs, ** kwargs))

    print(is_equal(r1, r2)[1])
    #print(is_equal(r1, r3)[1])
    print(is_equal(r1, r4)[1])
    #print(is_equal(r1, r5)[1])
    print(is_equal(r1, r6)[1])

    r7 = data_io.load_medical_data(tf.cast(path + '.nii.gz', tf.string), dtype = tf.uint8, ** kwargs)
    r8 = data_io.load_medical_data(tf.cast(path + path_infos + '.npy', tf.string), dtype = tf.uint8, ** kwargs)
    #r9 = data_io.load_medical_data(tf.cast(path + '.npz', tf.string), dtype = tf.uint8, ** kwargs)
    r10 = _to_dense(data_io.load_medical_data(tf.cast(path + '-sparse.npz', tf.string), ** sp_kwargs, ** kwargs))
    #r11 = data_io.load_medical_data(tf.cast(path + path_infos + '.tensor', tf.string), dtype = tf.uint8, ** kwargs)
    r12 = _to_dense(data_io.load_medical_data(tf.cast(path + path_infos + '.stensor', tf.string), ** sp_kwargs, ** kwargs))

    print(is_equal(r1, r7)[1])
    print(is_equal(r1, r8)[1])
    #print(is_equal(r1, r9)[1])
    print(is_equal(r1, r10)[1])
    #print(is_equal(r1, r11)[1])
    print(is_equal(r1, r12)[1])




Test with kwargs : {'slice_end': <tf.Tensor: shape=(), dtype=int32, numpy=6>}
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !

Test with kwargs : {'slice_start': <tf.Tensor: shape=(), dtype=int32, numpy=3>, 'slice_end': <tf.Tensor: shape=(), dtype=int32, numpy=6>}
5 out of the last 9 calls to <function load_medical_data at 0x7f60801ca3a0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and http

In [4]:
%timeit load_medical_data(path + '.nii.gz', dtype = tf.uint8, ** kwargs)
%timeit load_medical_data(path + path_infos + '.npy', dtype = tf.uint8, ** kwargs)
#%timeit load_medical_data(path + '.npz', dtype = tf.uint8, ** kwargs)
%timeit load_medical_data(path + '-sparse.npz', ** sp_kwargs, ** kwargs)
#%timeit load_medical_data(path + path_infos + '.tensor', dtype = tf.uint8, ** kwargs)
%timeit load_medical_data(path + path_infos + '.stensor', ** sp_kwargs, ** kwargs)
print()
%timeit load_medical_data(tf.cast(path + '.nii.gz', tf.string), dtype = tf.uint8, ** kwargs)
%timeit load_medical_data(tf.cast(path + path_infos + '.npy', tf.string), dtype = tf.uint8, ** kwargs)
#%timeit load_medical_data(tf.cast(path + '.npz', tf.string), dtype = tf.uint8, ** kwargs)
%timeit load_medical_data(tf.cast(path + '-sparse.npz', tf.string), ** sp_kwargs, ** kwargs)
#%timeit load_medical_data(tf.cast(path + path_infos + '.tensor', tf.string), dtype = tf.uint8, ** kwargs)
%timeit load_medical_data(tf.cast(path + path_infos + '.stensor', tf.string), ** sp_kwargs, ** kwargs)
print()
%timeit tf_load_medical_data(tf.cast(path + '.nii.gz', tf.string), dtype = tf.uint8, ** kwargs)
%timeit tf_load_medical_data(tf.cast(path + path_infos + '.npy', tf.string), dtype = tf.uint8, ** kwargs)
#%timeit tf_load_medical_data(tf.cast(path + '.npz', tf.string), dtype = tf.uint8, ** kwargs)
%timeit tf_load_medical_data(tf.cast(path + '-sparse.npz', tf.string), ** sp_kwargs, ** kwargs)
#%timeit tf_load_medical_data(tf.cast(path + path_infos + '.tensor', tf.string), dtype = tf.uint8, ** kwargs)
%timeit tf_load_medical_data(tf.cast(path + path_infos + '.stensor', tf.string), ** sp_kwargs, ** kwargs)


2.42 s ± 1.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.79 s ± 387 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
203 ms ± 575 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
79.9 ms ± 255 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

2.38 s ± 2.74 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.77 s ± 956 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
204 ms ± 774 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
79.7 ms ± 167 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

2.4 s ± 1.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.77 s ± 1.05 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
205 ms ± 770 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
66.5 ms ± 377 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
import importlib
import tensorflow as tf

from utils.med_utils import label_utils

importlib.reload(label_utils)

labels  = tf.cast(['a', 'b', 'c'], tf.string)
mapping = [None, ('a', 'b'), 'c', 'd']

matrix = label_utils.build_mapping(mapping, output_format = 'tf')
print(labels)
print(matrix)
match = labels[tf.newaxis, tf.newaxis, :] == matrix[:, :, tf.newaxis]
print(tf.reduce_any(match, axis = 1))

tf.Tensor([b'a' b'b' b'c'], shape=(3,), dtype=string)
tf.Tensor(
[[b'' b'']
 [b'a' b'b']
 [b'c' b'']
 [b'd' b'']], shape=(4, 2), dtype=string)
tf.Tensor(
[[False False False]
 [ True  True False]
 [False False  True]
 [False False False]], shape=(4, 3), dtype=bool)
